## 4. Modeling
### 4.1 Importing modules & loading data
### 4.2 Unsupervised clustering  
### 4.3 Conclusion & summary

### 4.1 Importing modules & loading data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [ ]:
features = car_acc.drop(['state', 'drvr_fatl_col_bmiles'], axis=1)
target = car_acc.drvr_fatl_col_bmiles.copy()

# Create a linear regression object
reg = linear_model.LinearRegression()

# Fit a multivariate linear regression model
reg.fit(features,target)
reg.score(features,target)
# Retrieve the regression coefficients
fit_coef = reg.coef_
fit_coef

### 4.2 Unsupervised clustering  
#### Clustering states by population, employeed, unemployment_rate, total_licensees, total_gun_deaths, gender, age, sectors, mentor_health_issues

In [ ]:
# A loop will be used to plot the explanatory power for up to 10 KMeans clusters
ks = range(1, 10)
inertias = []
for k in ks:
    # Initialize the KMeans object using the current number of clusters (k)
    km = KMeans(n_clusters=k, random_state=8)
    # Fit the scaled features to the KMeans object
    km.fit(features_scaled)
    # Append the inertia for `km` to the list of inertias
    inertias.append(km.inertia_)
    
# Plot the results in a line plot
plt.plot(ks, inertias, marker='o')

In [ ]:
# Create a KMeans object with 3 clusters, use random_state=8 
km = KMeans(n_clusters=3, random_state=8)

# Fit the data to the `km` object
km.fit(features_scaled)

# Create a scatter plot of the first two principal components
# and color it according to the KMeans cluster assignment 
plt.scatter(p_comp1, p_comp2, c=km.labels_)

In [ ]:
# Create a new column with the labels from the KMeans clustering
car_acc['cluster'] = km.labels_
display(car_acc.head())

# Reshape the DataFrame to the long format
melt_car = pd.melt(car_acc, id_vars=['cluster'],
                  var_name='measurement', value_name='percent',
                  value_vars=['perc_fatl_speed', 'perc_fatl_alcohol','perc_fatl_1st_time'])
display(melt_car.head())

# Create a violin plot splitting and coloring the results according to the km-clusters
sns.violinplot(melt_car['percent'], melt_car['measurement'], hue=melt_car['cluster'])